In [25]:
from langgraph.graph import StateGraph,START,END
from langchain_openai import ChatOpenAI
from typing import TypedDict,Annotated
from dotenv import load_dotenv
import os
from pydantic import BaseModel,Field 
import operator
load_dotenv() 

True

In [18]:
model=ChatOpenAI(model='gpt-4o-mini')



In [19]:
class EvaluationSchema(BaseModel):
    feedback:str=Field(description="detail feednack for the essay")
    score:int=Field(description='Score out of 10',ge=0,le=10)


In [20]:
structured_model=model.with_structured_output(EvaluationSchema)

In [21]:
essay= """ Curiosity is the spark that drives learning, creativity, and progress. From childhood, it inspires us to ask questions, explore the unknown, and challenge established ideas. Great discoveries in science, art, and technology often began with a simple “why” or “what if.” Curiosity also enriches daily life by helping us notice small details, appreciate diverse perspectives, and connect more deeply with others. In a rapidly changing world, it is curiosity that encourages adaptability and resilience. Unlike fleeting motivation, curiosity sustains lifelong growth, making it one of the most powerful traits a person can nurture."""

In [22]:
prompt = f"evaluate the language quality of following essay and provided a feedback and assign a score out of 10 \n  {essay}"

In [ ]:
structured_model.invoke(prompt)
# structured_model.invoke(prompt).feedback

EvaluationSchema(feedback='The essay presents a strong argument for the importance of curiosity in various aspects of life. It is well-structured with clear reasoning and insightful examples. The language used is articulate, making good use of vocabulary that enhances the message. However, the essay could benefit from a few more specific examples or anecdotes to further illustrate the points made and engage the reader more deeply. Additionally, some phrases could be varied to avoid repetition, making it more dynamic.', score=8)

In [28]:
class UPSCState(TypedDict):
    essay:str
    language_feedback:str
    analysis_feedback:str
    clarity_feedback:str
    overall_feedback:str
    individual_scores:Annotated[list[int],operator.add]
    avg_score:float

In [44]:
def evaluate_language(State:UPSCState)->UPSCState:
    prompt = f"evaluate the language quality of following essay and provided a feedback and assign a score out of 10 \n  {State['essay']}"
    output= structured_model.invoke(prompt)
    return {"language_feedback":output.feedback,"individual_scores":[output.score]}



In [48]:
def evaluate_analysis(State:UPSCState)->UPSCState:
    prompt = f"evaluate the depth of analysis from the following essay and provided a feedback and assign a score out of 10 \n  {State['essay']}"
    output= structured_model.invoke(prompt)
    return {"analysis_feedback":output.feedback,"individual_scores":[output.score]}

def evaluate_thought(State:UPSCState)->UPSCState:
    prompt = f"evaluate the clarity of thought from the following essay and provided a feedback and assign a score out of 10 \n  {State['essay']}"
    output= structured_model.invoke(prompt)
    return {"clarity_feedback":output.feedback,"individual_scores":[output.score]}

def final_evaluation(State:UPSCState)->UPSCState:
    prompt = f""" Based on the following feedback create a summarized feedback. 
    \n language feedback={State['language_feedback']} 
    \n analysis feedback= {State["analysis_feedback"]} 
    \n clarity feedback = {State["clarity_feedback"]}
    """
    overall_feedback= model.invoke(prompt).content

    avg_score=sum(State["individual_scores"] )/len(State["individual_scores"])
    return {"overall_feedback":overall_feedback,"avg_score":[avg_score]}




In [49]:
graph=StateGraph(UPSCState)
graph.add_node('evaluate_language',evaluate_language)
graph.add_node('evaluate_analysis',evaluate_analysis)
graph.add_node('evaluate_thought',evaluate_thought)
graph.add_node('final_evaluation',final_evaluation)

graph.add_edge(START,"evaluate_language")
graph.add_edge(START,"evaluate_analysis")
graph.add_edge(START,"evaluate_thought")


graph.add_edge("evaluate_thought","final_evaluation")
graph.add_edge("evaluate_analysis","final_evaluation")
graph.add_edge("evaluate_thought","final_evaluation")


graph.add_edge("final_evaluation",END)


workflow=graph.compile()

In [50]:
initial_state={"essay":essay}
result=workflow.invoke(initial_state)

In [51]:
print(result)

{'essay': ' Curiosity is the spark that drives learning, creativity, and progress. From childhood, it inspires us to ask questions, explore the unknown, and challenge established ideas. Great discoveries in science, art, and technology often began with a simple “why” or “what if.” Curiosity also enriches daily life by helping us notice small details, appreciate diverse perspectives, and connect more deeply with others. In a rapidly changing world, it is curiosity that encourages adaptability and resilience. Unlike fleeting motivation, curiosity sustains lifelong growth, making it one of the most powerful traits a person can nurture.', 'language_feedback': "The essay is well-structured and presents a clear argument regarding the importance of curiosity. The language used is descriptive and engaging, effectively conveying the idea that curiosity is fundamental to personal growth and societal advancement. However, the essay could benefit from more varied sentence structures to enhance its